In [ ]:
from xml.etree.ElementPath import prepare_descendant

import pyodbc
import pandas as pd
import sqlite3

In [27]:
sales_conn = sqlite3.connect("DEDS_W2_Hulpmiddelen/go_sales_train.sqlite")
crm_conn = sqlite3.connect("DEDS_W2_Hulpmiddelen/go_crm_train.sqlite")
staff_conn = sqlite3.connect("DEDS_W2_Hulpmiddelen/go_staff_train.sqlite")

In [33]:


DB = {
    'servername': r'LAPTOP-PRQNR6SS\SQLEXPRESS',
    'database': 'SDM'
}

export_conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    f"SERVER={DB['servername']};"
    f"DATABASE={DB['database']};"
    "Trusted_Connection=yes;"
)

export_cursor = export_conn.cursor()
print("Succesvol verbonden met SQL Server!")


Succesvol verbonden met SQL Server!


In [34]:
sql_query = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name"
pd.read_sql_query(sql_query, sales_conn)

,name
0,country
1,order_details
2,order_header
3,order_method
4,product
5,product_line
6,product_type
7,retailer_site
8,return_reason
9,returned_item


# Country

In [52]:


# Data ophalen uit sales database
sales_query = "SELECT COUNTRY_CODE, COUNTRY, LANGUAGE, CURRENCY_NAME FROM country"
sales_country = pd.read_sql_query(sales_query, sales_conn)

# Data ophalen uit CRM database
crm_query = "SELECT COUNTRY_CODE, COUNTRY_EN, FLAG_IMAGE, SALES_TERRITORY_CODE FROM country"
crm_country = pd.read_sql_query(crm_query, crm_conn)

# Merge de data op COUNTRY_CODE
merged_data = pd.merge(sales_country, crm_country, on="COUNTRY_CODE", how="outer")

# Kolommen herschikken voor invoer in SSMS
merged_data = merged_data[["COUNTRY_CODE", "COUNTRY", "LANGUAGE", "CURRENCY_NAME", "FLAG_IMAGE", "SALES_TERRITORY_CODE"]]

merged_data



,COUNTRY_CODE,COUNTRY,LANGUAGE,CURRENCY_NAME,FLAG_IMAGE,SALES_TERRITORY_CODE
0,1,France,EN,francs,F01,6
1,2,Germany,EN,marks,F02,6
2,3,United States,EN,dollars,F03,1
3,4,Canada,EN,dollars,F04,1
4,5,Austria,EN,schillings,F05,7
5,6,Italy,EN,lira,F06,7
6,7,Netherlands,EN,guilders,F07,5
7,8,Switzerland,EN,francs,F08,6
8,9,United Kingdom,EN,pounds,F09,6
9,10,Sweden,EN,krona,F10,5


In [53]:

for index, row in merged_data.iterrows():
    try:
        query = """
        INSERT INTO country (COUNTRY_CODE, COUNTRY, LANGUAGE, CURRENCY_NAME, FLAG_IMAGE, SALES_TERRITORY_CODE)
        VALUES (?, ?, ?, ?, ?, ?)
        """
        export_cursor.execute(query, row["COUNTRY_CODE"], row["COUNTRY"], row["LANGUAGE"], row["CURRENCY_NAME"], row["FLAG_IMAGE"], row["SALES_TERRITORY_CODE"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")



Data succesvol geïmporteerd naar SSMS!


# sales_territory


In [51]:

sales_territory = pd.read_sql_query("SELECT SALES_TERRITORY_CODE, TERRITORY_NAME_EN FROM sales_territory", crm_conn)

for index, row in sales_territory.iterrows():
    try:
        query = """
        INSERT INTO sales_territory (SALES_TERRITORY_CODE, TERRITORY_NAME_EN) VALUES (?, ?)
        """
        export_cursor.execute(query, row["SALES_TERRITORY_CODE"], row["TERRITORY_NAME_EN"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")


Data succesvol geïmporteerd naar SSMS!


# Order_method


In [55]:
order_method = pd.read_sql_query("SELECT ORDER_METHOD_CODE, ORDER_METHOD_EN FROM order_method", sales_conn)

for index, row in order_method.iterrows():
    try:
        query = "INSERT INTO order_method (ORDER_METHOD_CODE, ORDER_METHOD_EN) VALUES (?, ?)"
        export_cursor.execute(query, row["ORDER_METHOD_CODE"], row["ORDER_METHOD_EN"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")

Data succesvol geïmporteerd naar SSMS!


# Product_Line


In [56]:
product_line = pd.read_sql_query("SELECT PRODUCT_LINE_CODE, PRODUCT_LINE_EN FROM product_line", sales_conn)

for index, row in product_line.iterrows():
    try:
        query = "INSERT INTO product_line (PRODUCT_LINE_CODE, PRODUCT_LINE_EN) VALUES (?, ?)"
        export_cursor.execute(query, row["PRODUCT_LINE_CODE"], row["PRODUCT_LINE_EN"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")



Data succesvol geïmporteerd naar SSMS!


# Retailer_segment


In [58]:
retailer_segment = pd.read_sql_query("SELECT SEGMENT_CODE, LANGUAGE, SEGMENT_NAME, SEGMENT_DESCRIPTION FROM retailer_segment", crm_conn)

for index, row in retailer_segment.iterrows():
    try:
        query = "INSERT INTO retailer_segment (SEGMENT_CODE, LANGUAGE, SEGMENT_NAME, SEGMENT_DESCRIPTION) VALUES (?, ?, ?, ?)"
        export_cursor.execute(query, row["SEGMENT_CODE"], row["LANGUAGE"], row["SEGMENT_NAME"], row["SEGMENT_DESCRIPTION"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")

Data succesvol geïmporteerd naar SSMS!


# Retailer_type

In [59]:
retailer_type = pd.read_sql_query("SELECT RETAILER_TYPE_CODE, RETAILER_TYPE_EN FROM retailer_type", crm_conn)

for index, row in retailer_type.iterrows():
    try:
        query = "INSERT INTO retailer_type (RETAILER_TYPE_CODE, RETAILER_TYPE_EN) VALUES (?, ?)"
        export_cursor.execute(query, row["RETAILER_TYPE_CODE"], row["RETAILER_TYPE_EN"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")

Data succesvol geïmporteerd naar SSMS!


# satisfaction_type

In [63]:
satisfaction_type = pd.read_sql_query("SELECT SATISFACTION_TYPE_CODE, SATISFACTION_TYPE_DESCRIPTION FROM satisfaction_type", staff_conn)

for index, row in satisfaction_type.iterrows():
    try:
        query = "INSERT INTO satisfaction_type (SATISFACTION_TYPE_CODE, SATISFACTION_TYPE_DESCRIPTION) VALUES (?, ?)"
        export_cursor.execute(query, row["SATISFACTION_TYPE_CODE"], row["SATISFACTION_TYPE_DESCRIPTION"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")

Data succesvol geïmporteerd naar SSMS!


# course

In [62]:
course = pd.read_sql_query("SELECT COURSE_CODE, COURSE_DESCRIPTION FROM course", staff_conn)

for index, row in course.iterrows():
    try:
        query = "INSERT INTO course (COURSE_CODE, COURSE_DESCRIPTION) VALUES (?, ?)"
        export_cursor.execute(query, row["COURSE_CODE"], row["COURSE_DESCRIPTION"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")

Data succesvol geïmporteerd naar SSMS!


# age_group

In [61]:
age_group = pd.read_sql_query("SELECT AGE_GROUP_CODE, UPPER_AGE, LOWER_AGE FROM age_group", crm_conn)

for index, row in age_group.iterrows():
    try:
        query = "INSERT INTO age_group (AGE_GROUP_CODE, UPPER_AGE, LOWER_AGE) VALUES (?, ?, ?)"
        export_cursor.execute(query, int(row["AGE_GROUP_CODE"]), int(row["UPPER_AGE"]), int(row["LOWER_AGE"]))
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")

Data succesvol geïmporteerd naar SSMS!


# return_reason

In [64]:
return_reason = pd.read_sql_query("SELECT RETURN_REASON_CODE, RETURN_DESCRIPTION_EN FROM return_reason", sales_conn)

for index, row in return_reason.iterrows():
    try:
        query = "INSERT INTO return_reason (RETURN_REASON_CODE, RETURN_DESCRIPTION_EN) VALUES (?, ?)"
        export_cursor.execute(query, row["RETURN_REASON_CODE"], row["RETURN_DESCRIPTION_EN"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")

Data succesvol geïmporteerd naar SSMS!


# sales_branch

In [65]:
sales_branch = pd.read_sql_query("SELECT SALES_BRANCH_CODE, ADDRESS1, ADDRESS2, CITY, REGION, COUNTRY_CODE FROM sales_branch", sales_conn)

for index, row in sales_branch.iterrows():
    try:
        query = "INSERT INTO sales_branch (SALES_BRANCH_CODE, ADDRESS1, ADDRESS2, CITY, REGION, COUNTRY_CODE) VALUES (?, ?, ?, ?, ?, ?)"
        export_cursor.execute(query, row["SALES_BRANCH_CODE"], row["ADDRESS1"], row["ADDRESS2"], row["CITY"], row["REGION"], row["COUNTRY_CODE"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")

Data succesvol geïmporteerd naar SSMS!


# product_type

In [66]:
product_type = pd.read_sql_query("SELECT PRODUCT_TYPE_CODE, PRODUCT_LINE_CODE, PRODUCT_TYPE_EN FROM product_type", sales_conn)

for index, row in product_type.iterrows():
    try:
        query = "INSERT INTO product_type (PRODUCT_TYPE_CODE, PRODUCT_LINE_CODE, PRODUCT_TYPE_EN) VALUES (?, ?, ?)"
        export_cursor.execute(query, row["PRODUCT_TYPE_CODE"], row["PRODUCT_LINE_CODE"], row["PRODUCT_TYPE_EN"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")

Data succesvol geïmporteerd naar SSMS!


# retailer_headquarter

In [67]:
retailer_headquarters = pd.read_sql_query("SELECT RETAILER_CODEMR, RETAILER_NAME, ADDRESS1, ADDRESS2, CITY, REGION, COUNTRY_CODE, PHONE, FAX, SEGMENT_CODE FROM retailer_headquarters", crm_conn)

for index, row in retailer_headquarters.iterrows():
    try:
        query = "INSERT INTO retailer_headquarters (RETAILER_CODEMR, RETAILER_NAME, ADDRESS1, ADDRESS2, CITY, REGION, COUNTRY_CODE, PHONE, FAX, SEGMENT_CODE) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
        export_cursor.execute(query, row["RETAILER_CODEMR"], row["RETAILER_NAME"], row["ADDRESS1"], row["ADDRESS2"], row["CITY"], row["REGION"], row["COUNTRY_CODE"], row["PHONE"], row["FAX"], row["SEGMENT_CODE"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")

Data succesvol geïmporteerd naar SSMS!


# sales_staff

In [76]:
# Data ophalen uit sales database
sales_query = "SELECT SALES_STAFF_CODE, FIRST_NAME, LAST_NAME, POSITION_EN, WORK_PHONE, EXTENSION, FAX, EMAIL, DATE_HIRED, SALES_BRANCH_CODE FROM sales_staff"
sales_staff = pd.read_sql_query(sales_query, sales_conn)

# Data ophalen uit CRM database
staff_query = "SELECT SALES_STAFF_CODE, MANAGER_CODE FROM sales_staff"
staff_staff = pd.read_sql_query(staff_query, staff_conn)

# Merge de data op SALES_STAFF_CODE
merged_staff = pd.merge(sales_staff, staff_staff, on="SALES_STAFF_CODE", how="outer")

# Kolommen herschikken voor invoer in SSMS
merged_staff = merged_staff[["SALES_STAFF_CODE", "FIRST_NAME", "LAST_NAME", "POSITION_EN", "WORK_PHONE", "EXTENSION", "FAX", "EMAIL", "DATE_HIRED", "SALES_BRANCH_CODE", "MANAGER_CODE"]]

merged_staff


,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION_EN,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE,MANAGER_CODE
0,4,Denis,Pagé,Branch Manager,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6,18
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6,4
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6,4
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6,4
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13,14
...,...,...,...,...,...,...,...,...,...,...,...
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1340.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38,119
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38,119
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38,119
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39,18


In [87]:
import numpy as np

for index, row in merged_staff.iterrows():
    try:
        query = """
        INSERT INTO sales_staff (SALES_STAFF_CODE, FIRST_NAME, LAST_NAME, POSITION_EN, WORK_PHONE, EXTENSION, FAX, EMAIL, DATE_HIRED, SALES_BRANCH_CODE)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        export_cursor.execute(
            query,
            row["SALES_STAFF_CODE"],
            row["FIRST_NAME"],
            row["LAST_NAME"],
            row["POSITION_EN"],
            str(row["WORK_PHONE"]) if not pd.isna(row["WORK_PHONE"]) else None,
            int(row["EXTENSION"]) if not pd.isna(row["EXTENSION"]) else None,
            str(row["FAX"]) if not pd.isna(row["FAX"]) else None,
            row["EMAIL"],
            str(row["DATE_HIRED"]) if not pd.isna(row["DATE_HIRED"]) else None,
            row["SALES_BRANCH_CODE"]
        )
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")


Fout bij query: 
        INSERT INTO sales_staff (SALES_STAFF_CODE, FIRST_NAME, LAST_NAME, POSITION_EN, WORK_PHONE, EXTENSION, FAX, EMAIL, DATE_HIRED, SALES_BRANCH_CODE)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        
Foutmelding: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK__sales_st__F86DFD196A6052AF'. Cannot insert duplicate key in object 'dbo.sales_staff'. The duplicate key value is (4). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")
Fout bij query: 
        INSERT INTO sales_staff (SALES_STAFF_CODE, FIRST_NAME, LAST_NAME, POSITION_EN, WORK_PHONE, EXTENSION, FAX, EMAIL, DATE_HIRED, SALES_BRANCH_CODE)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        
Foutmelding: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK__sales_st__F86DFD196A6052AF'. Cannot insert duplicate key in object '

In [88]:
# Stap 2: Daarna MANAGER_CODE bijwerken
for index, row in merged_staff.iterrows():
    if row["MANAGER_CODE"] is not None:
        try:
            query = """
            UPDATE sales_staff
            SET MANAGER_CODE = ?
            WHERE SALES_STAFF_CODE = ?
            """
            export_cursor.execute(query, row["MANAGER_CODE"], row["SALES_STAFF_CODE"])
        except pyodbc.Error as e:
            print(f"Fout bij query: {query}")
            print(f"Foutmelding: {e}")

export_conn.commit()

print("Data succesvol geïmporteerd naar SSMS!")

Data succesvol geïmporteerd naar SSMS!


# retailer


In [96]:
retailer = pd.read_sql_query("SELECT RETAILER_CODE, RETAILER_CODEMR, COMPANY_NAME, RETAILER_TYPE_CODE FROM retailer", crm_conn)

for index, row in retailer.iterrows():
    try:
        retailer_code_mr = None if pd.isna(row["RETAILER_CODEMR"]) else int(row["RETAILER_CODEMR"])

        query = "INSERT INTO retailer (RETAILER_CODE, RETAILER_CODEMR, COMPANY_NAME, RETAILER_TYPE_CODE) VALUES (?, ?, ?, ?)"
        export_cursor.execute(query, row["RETAILER_CODE"], retailer_code_mr, row["COMPANY_NAME"], row["RETAILER_TYPE_CODE"])
    except pyodbc.Error as e:
        print(f"Fout bij rij: {row}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")


Data succesvol geïmporteerd naar SSMS!


# retailer_site

In [102]:
retailer_site = pd.read_sql_query("SELECT RETAILER_SITE_CODE, RETAILER_CODE, ADDRESS1, ADDRESS2, CITY, REGION, COUNTRY_CODE, ACTIVE_INDICATOR FROM retailer_site", sales_conn)

for index, row in retailer_site.iterrows():
    try:
        query = "INSERT INTO retailer_site (RETAILER_SITE_CODE, RETAILER_CODE, ADDRESS1, ADDRESS2, CITY, REGION, COUNTRY_CODE, ACTIVE_INDICATOR) VALUES (?, ?, ?, ?, ?, ?, ?, ?)"
        export_cursor.execute(query, row["RETAILER_SITE_CODE"], row["RETAILER_CODE"], row["ADDRESS1"], row["ADDRESS2"], row["CITY"], row["REGION"], row["COUNTRY_CODE"], row["ACTIVE_INDICATOR"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")


Data succesvol geïmporteerd naar SSMS!


# product

In [103]:
product = pd.read_sql_query("SELECT PRODUCT_NUMBER, INTRODUCTION_DATE, PRODUCT_TYPE_CODE, PRODUCTION_COST, MARGIN, PRODUCT_IMAGE, LANGUAGE, PRODUCT_NAME, DESCRIPTION FROM product", sales_conn)

for index, row in product.iterrows():
    try:
        query = "INSERT INTO product (PRODUCT_NUMBER, INTRODUCTION_DATE, PRODUCT_TYPE_CODE, PRODUCTION_COST, MARGIN, PRODUCT_IMAGE, LANGUAGE, PRODUCT_NAME, DESCRIPTION) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)"
        export_cursor.execute(query, row["PRODUCT_NUMBER"], row["INTRODUCTION_DATE"], row["PRODUCT_TYPE_CODE"], row["PRODUCTION_COST"], row["MARGIN"], row["PRODUCT_IMAGE"], row["LANGUAGE"], row["PRODUCT_NAME"], row["DESCRIPTION"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")


Data succesvol geïmporteerd naar SSMS!


# satisfaction

In [110]:
satisfaction = pd.read_sql_query("SELECT YEAR, SALES_STAFF_CODE, SATISFACTION_TYPE_CODE FROM satisfaction", staff_conn)

for index, row in satisfaction.iterrows():
    try:
        query = "INSERT INTO satisfaction (YEAR, SALES_STAFF_CODE, SATISFACTION_TYPE_CODE) VALUES (?, ?, ?)"
        export_cursor.execute(query, int(row["YEAR"]), int(row["SALES_STAFF_CODE"]), int(row["SATISFACTION_TYPE_CODE"]))
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")




Data succesvol geïmporteerd naar SSMS!


# training

In [109]:
training = pd.read_sql_query("SELECT YEAR, SALES_STAFF_CODE, COURSE_CODE FROM training", staff_conn)

for index, row in training.iterrows():
    try:
        query = "INSERT INTO training (YEAR, SALES_STAFF_CODE, COURSE_CODE) VALUES (?, ?, ?)"
        export_cursor.execute(query, int(row["YEAR"]), int(row["SALES_STAFF_CODE"]), int(row["COURSE_CODE"]))
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")

Data succesvol geïmporteerd naar SSMS!


# order_header

In [111]:
order_header = pd.read_sql_query("SELECT ORDER_NUMBER, RETAILER_NAME, RETAILER_SITE_CODE, SALES_STAFF_CODE, SALES_BRANCH_CODE, ORDER_DATE, ORDER_METHOD_CODE FROM order_header", sales_conn)

for index, row in order_header.iterrows():
    try:
        query = "INSERT INTO order_header (ORDER_NUMBER, RETAILER_NAME, RETAILER_SITE_CODE, SALES_STAFF_CODE, SALES_BRANCH_CODE, ORDER_DATE, ORDER_METHOD_CODE) VALUES (?, ?, ?, ?, ?, ?, ?)"
        export_cursor.execute(query, row["ORDER_NUMBER"], row["RETAILER_NAME"], row["RETAILER_SITE_CODE"], row["SALES_STAFF_CODE"], row["SALES_BRANCH_CODE"], row["ORDER_DATE"], row["ORDER_METHOD_CODE"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")




Data succesvol geïmporteerd naar SSMS!


In [125]:
try:
    query = """
    UPDATE order_header
    SET RETAILER_CONTACT_CODE = (
        SELECT rc.RETAILER_CONTACT_CODE
        FROM retailer_contact rc
        JOIN retailer_site rs ON rc.RETAILER_SITE_CODE = rs.RETAILER_SITE_CODE
        WHERE rs.RETAILER_SITE_CODE = order_header.RETAILER_SITE_CODE
    )
    WHERE EXISTS (
        SELECT 1
        FROM retailer_contact rc
        JOIN retailer_site rs ON rc.RETAILER_SITE_CODE = rs.RETAILER_SITE_CODE
        WHERE rs.RETAILER_SITE_CODE = order_header.RETAILER_SITE_CODE
    )
    """
    export_cursor.execute(query)
    export_conn.commit()
    print("RETAILER_CONTACT_CODE succesvol geüpdatet in order_header!")
except pyodbc.Error as e:
    print(f"Fout bij query: {query}")
    print(f"Foutmelding: {e}")


RETAILER_CONTACT_CODE succesvol geüpdatet in order_header!


# sales_branch

In [113]:
sales_branch = pd.read_sql_query("SELECT SALES_BRANCH_CODE, ADDRESS1, ADDRESS2, CITY, REGION, COUNTRY_CODE FROM sales_branch", sales_conn)

for index, row in sales_branch.iterrows():
    try:
        query = "INSERT INTO sales_branch (SALES_BRANCH_CODE, ADDRESS1, ADDRESS2, CITY, REGION, COUNTRY_CODE) VALUES (?, ?, ?, ?, ?, ?)"
        export_cursor.execute(query, row["SALES_BRANCH_CODE"], row["ADDRESS1"], row["ADDRESS2"], row["CITY"], row["REGION"], row["COUNTRY_CODE"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")


,SALES_BRANCH_CODE,ADDRESS1,ADDRESS2,CITY,REGION,COUNTRY_CODE
0,6,"75, rue du Faubourg St-Honoré",None,Paris,None,1
1,7,"Piazza Duomo, 1",None,Milano,None,6
2,9,Singelgravenplein 4,4e verdieping,Amsterdam,Noord-Holland,7
3,13,Schwabentor 35,None,Hamburg,None,2
4,14,Leopoldstraße 36,None,München,None,2
5,15,Isafjordsgatan 30 C,None,Kista,None,10
6,17,"7800, 756 - 6th Avenue. S.W.",None,Calgary,Alberta,4
7,18,789 Yonge Street,None,Toronto,Ontario,4
8,19,1288 Dorchester Avenue,None,Boston,Massachusetts,3
9,20,299 Yale Avenue,None,Seattle,Washington,3


# order_details


In [114]:
order_details = pd.read_sql_query("SELECT ORDER_DETAIL_CODE, ORDER_NUMBER, PRODUCT_NUMBER, QUANTITY, UNIT_COST, UNIT_PRICE, UNIT_SALE_PRICE FROM order_details", sales_conn)

for index, row in order_details.iterrows():
    try:
        query = "INSERT INTO order_details (ORDER_DETAIL_CODE, ORDER_NUMBER, PRODUCT_NUMBER, QUANTITY, UNIT_COST, UNIT_PRICE, UNIT_SALE_PRICE) VALUES (?, ?, ?, ?, ?, ?, ?)"
        export_cursor.execute(query, row["ORDER_DETAIL_CODE"], row["ORDER_NUMBER"], row["PRODUCT_NUMBER"], row["QUANTITY"], row["UNIT_COST"], row["UNIT_PRICE"], row["UNIT_SALE_PRICE"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")


Data succesvol geïmporteerd naar SSMS!


# invertory_level

In [138]:
inventory_level = pd.read_csv("DEDS_W2_Hulpmiddelen/inventory_levels_train.csv")

inventory_level = inventory_level.drop(columns="Unnamed: 0", errors="ignore")

inventory_level




,INVENTORY_YEAR,INVENTORY_MONTH,PRODUCT_NUMBER,INVENTORY_COUNT
0,2023,4,48,1932
1,2023,4,49,1400
2,2023,4,50,21705
3,2023,4,51,9710
4,2023,4,52,5616
...,...,...,...,...
3538,2023,4,43,6147
3539,2023,4,44,5822
3540,2023,4,45,1676
3541,2023,4,46,1802


In [141]:
for index, row in inventory_level.iterrows():
    try:
        query = """
        INSERT INTO inventory_levels_train (INVENTORY_YEAR, INVENTORY_MONTH, PRODUCT_NUMBER, INVENTORY_COUNT)
        VALUES (?, ?, ?, ?)
        """
        export_cursor.execute(query, int(row["INVENTORY_YEAR"]), int(row["INVENTORY_MONTH"]), int(row["PRODUCT_NUMBER"]), int(row["INVENTORY_COUNT"]))
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

# Commit de wijzigingen en sluit de verbinding
export_conn.commit()

print("Data succesvol geïmporteerd naar SSMS!")

Data succesvol geïmporteerd naar SSMS!


# product_forecast

In [142]:
product_forecast = pd.read_csv("DEDS_W2_Hulpmiddelen/product_forecast_train.csv")

product_forecast

,PRODUCT_NUMBER,YEAR,MONTH,EXPECTED_VOLUME
0,45,2023,12,90
1,46,2023,12,84
2,47,2023,12,468
3,48,2023,12,91
4,49,2023,12,35
...,...,...,...,...
3524,37,2022,1,60
3525,38,2022,1,51
3526,39,2022,1,235
3527,40,2022,1,568


In [144]:


for index, row in product_forecast.iterrows():
    try:
        query = "INSERT INTO product_forecast_train (PRODUCT_NUMBER, YEAR, MONTH, EXPECTED_VOLUME) VALUES (?, ?, ?, ?)"
        export_cursor.execute(query, int(row["PRODUCT_NUMBER"]), int(row["YEAR"]), int(row["MONTH"]), int(row["EXPECTED_VOLUME"]))
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")


Data succesvol geïmporteerd naar SSMS!


# retailer_contact

In [120]:
retailer_contact = pd.read_sql_query("SELECT RETAILER_CONTACT_CODE, RETAILER_SITE_CODE, FIRST_NAME, LAST_NAME, JOB_POSITION_EN, EXTENSION, FAX, E_MAIL, GENDER FROM retailer_contact", crm_conn)

for index, row in retailer_contact.iterrows():
    try:
        retailer_extension =None if pd.isna(row["EXTENSION"]) else int(row["EXTENSION"])

        query = "INSERT INTO retailer_contact (RETAILER_CONTACT_CODE, RETAILER_SITE_CODE, FIRST_NAME, LAST_NAME, JOB_POSITION_EN, EXTENSION, FAX, E_MAIL, GENDER) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)"

        export_cursor.execute(query, row["RETAILER_CONTACT_CODE"], row["RETAILER_SITE_CODE"], row["FIRST_NAME"], row["LAST_NAME"], row["JOB_POSITION_EN"], retailer_extension, row["FAX"], row["E_MAIL"], row["GENDER"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")




Data succesvol geïmporteerd naar SSMS!


,RETAILER_CONTACT_CODE,RETAILER_SITE_CODE,FIRST_NAME,LAST_NAME,JOB_POSITION_EN,EXTENSION,FAX,E_MAIL,GENDER
0,4,19,Gilles,Rapier,Site Manager,NaN,1 (613) 740-6701,GRapier@f76.net,M
1,5,23,Walter,Baker,Site Manager,NaN,1 (514) 743-7985,WBaker@maxsnetinc.com,M
2,6,4,Marie,Turcotte,Assistant Purchaser,635.0,1 (514) 747-2582,MTurcotte@cybserv19.com,F
3,7,2,Mark,Myers,Assistant Purchaser,NaN,1 (514) 741-2277,MMyers@rocksteadyitl.com,M
4,9,10,Jack,Marshall,Site Manager,NaN,1 (604) 682-4523,JMarshall@vipce2k.com,M
...,...,...,...,...,...,...,...,...,...
386,458,439,Manfred,Herwig,Chief Purchaser,NaN,+(43) 12 72 92 56,MHerwig@ausglobetrotter.net,M
387,459,440,Anna-Maria,Pump,Stock Manager,NaN,+(43) 73 57 10 40,APump@ausglobetrotter.net,F
388,460,441,Franziska,Resch,Site Assistant Manager,NaN,+(43) 51 28 28 08,FResch@ausglobetrotter.net,F
389,461,442,Peter,Werner,Site Manager,NaN,+(43) 54 44 78 79,PWerner@ausglobetrotter.net,M


# sales_demographic

In [122]:
sales_demographic = pd.read_sql_query("SELECT DEMOGRAPHIC_CODE, RETAILER_CODEMR, AGE_GROUP_CODE, SALES_PERCENT FROM sales_demographic", crm_conn)

for index, row in sales_demographic.iterrows():
    try:
        query = "INSERT INTO sales_demographic (DEMOGRAPHIC_CODE, RETAILER_CODEMR, AGE_GROUP_CODE, SALES_PERCENT) VALUES (?, ?, ?, ?)"
        export_cursor.execute(query, row["DEMOGRAPHIC_CODE"], row["RETAILER_CODEMR"], row["AGE_GROUP_CODE"], row["SALES_PERCENT"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")


Data succesvol geïmporteerd naar SSMS!


# returned_item

In [123]:
returned_item = pd.read_sql_query("SELECT RETURN_CODE, RETURN_DATE, ORDER_DETAIL_CODE, RETURN_REASON_CODE, RETURN_QUANTITY FROM returned_item", sales_conn)

for index, row in returned_item.iterrows():
    try:
        query = "INSERT INTO returned_item (RETURN_CODE, RETURN_DATE, ORDER_DETAIL_CODE, RETURN_REASON_CODE, RETURN_QUANTITY) VALUES (?, ?, ?, ?, ?)"
        export_cursor.execute(query, row["RETURN_CODE"], row["RETURN_DATE"], row["ORDER_DETAIL_CODE"], row["RETURN_REASON_CODE"], row["RETURN_QUANTITY"])
    except pyodbc.Error as e:
        print(f"Fout bij query: {query}")
        print(f"Foutmelding: {e}")

export_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")


Data succesvol geïmporteerd naar SSMS!


In [124]:
order_header = pd.read_sql_query("SELECT ORDER_NUMBER, RETAILER_NAME, RETAILER_SITE_CODE, SALES_STAFF_CODE, SALES_BRANCH_CODE, ORDER_DATE, ORDER_METHOD_CODE FROM order_header", sales_conn)

order_header

,ORDER_NUMBER,RETAILER_NAME,RETAILER_SITE_CODE,SALES_STAFF_CODE,SALES_BRANCH_CODE,ORDER_DATE,ORDER_METHOD_CODE
0,1153,Ultra Sports,4,50,18,2022-04-14,7
1,1154,Ultra Sports,4,50,18,2022-10-01,4
2,1155,Ultra Sports,5,49,18,2022-04-21,7
3,1156,Ultra Sports,5,49,18,2022-09-09,5
4,1157,Ultra Sports,6,50,18,2022-04-10,2
...,...,...,...,...,...,...,...
4779,9475,Preben's T°y,338,29,15,2022-01-13,4
4780,9476,Preben's T°y,336,29,15,2023-01-02,5
4781,9477,Preben's T°y,338,29,15,2023-01-18,2
4782,9478,Preben's T°y,336,27,15,2024-01-15,7
